In [1]:
from time import sleep

In [2]:
from serial_interface import *

In [3]:
%matplotlib ipympl

In [4]:
import matplotlib.pyplot as plt

In [5]:
from pydgilib_extra import *

In [6]:
dgilib_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\dgilib.dll"

In [7]:
fig = plt.figure(figsize=(10, 6))

FigureCanvasNbAgg()

In [8]:
data = []

config_dict = {
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": True,
    "fig": fig,
    "verbose": 0,
}

fig.clf()
pin = 3

with DGILibExtra(dgilib_path, **config_dict) as dgilib, SAML11() as board:
    dgilib.device_reset()
    dgilib.logger_start()
    print(board.get_status())
    board.send_command("?", True)
    board.send_command("STA", True)
    board.set_debug(True)
    board.send_command("STA", True)
    board.set_debug(False)
    board.send_command("STA", True)
    board.send_command("TAE", True)
    for i in range(100):
        dgilib.update_callback()
#     sleep(1)
    dgilib.logger_stop()
#     gpio_augment_edges(dgilib.data[INTERFACE_GPIO])
#     dgilib.ax.plot(*dgilib.power_filter_by_pin(0))
#     dgilib.ax.plot(*dgilib.power_filter_by_pin(1))
    dgilib.ax.plot(*dgilib.power_filter_by_pin(2))
    dgilib.ax.plot(*dgilib.power_filter_by_pin(3))
    data = dgilib.data

DeviceReturnError: connect returned: 17